# Tarea de Topicos Avanzados de Inteligencia Artificial

## Dataset Utilizada: https://www.kaggle.com/boltzmannbrain/nab

### Contexto:
    Numenta Anomaly Benchmark (NAB) es un nuevo punto de referencia para evaluar algoritmos para la detección de anomalías en aplicaciones de transmisión en línea.
    Se compone de más de 50 archivos de datos de series temporales artificiales y del mundo real etiquetados, además de un mecanismo de puntuación novedoso diseñado
    para aplicaciones en tiempo real. Todos los datos y el código son completamente de código abierto, con una amplia documentación y un cuadro de indicadores de 
    algoritmos de detección de anomalías: github.com/numenta/NAB. El conjunto de datos completo se incluye aquí, pero vaya al repositorio para obtener detalles 
    sobre cómo evaluar los algoritmos de detección de anomalías en NAB.
    
    
## Para este proyecto se utilizo la carpeta realAWSCloudwatch/
    Métricas del servidor de AWS recopiladas por el servicio AmazonCloudwatch. Las métricas de ejemplo incluyen Utilización de CPU, Bytes de red en y Bytes de lectura de disco. 



# 1. Importando Librerías

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

        
import plotly.express as px
import altair as alt
from sklearn.ensemble import IsolationForest
import plotly.graph_objects as go

# 2. Entendiendo la base de datos

In [ ]:
cloudwatch_df = pd.read_csv("/kaggle/input/nab/realAWSCloudwatch/realAWSCloudwatch/ec2_cpu_utilization_53ea38.csv")
cloudwatch_df.head()

In [ ]:
cloudwatch_df.shape

In [ ]:
cloudwatch_df.info()

In [ ]:
cloudwatch_df.describe()

# 3. Preprocessing/ feature engineering

In [ ]:
cloudwatch_df['marcaTiempo'] = pd.to_datetime(cloudwatch_df['timestamp'])
cloudwatch_df.info()

In [ ]:
cloudwatch_df['año'] = cloudwatch_df['marcaTiempo'].apply(lambda x: x.year)
cloudwatch_df['mes'] = cloudwatch_df['marcaTiempo'].apply(lambda x: x.month)
cloudwatch_df['dia'] = cloudwatch_df['marcaTiempo'].apply(lambda x: x.day)
cloudwatch_df['diaSemana'] = cloudwatch_df['marcaTiempo'].apply(lambda x: x.weekday())
cloudwatch_df['hora'] = cloudwatch_df['marcaTiempo'].apply(lambda x: x.hour)

cloudwatch_df = cloudwatch_df[['marcaTiempo', 'año', 'mes', 'dia', 'diaSemana', 'hora', 'value']]
cloudwatch_df_esp = cloudwatch_df.rename(columns={'value': 'valor'})
# Weekday starts from Monday
print(f'{cloudwatch_df.marcaTiempo[0]} con día de la semana {cloudwatch_df.diaSemana[0]} es {cloudwatch_df.marcaTiempo[0].strftime("%A")}.\n')

cloudwatch_df_esp.head()

In [ ]:
cloudwatch_df_esp.describe()

# 4. Exploratory Data Analysis

In [ ]:
fig = px.line(cloudwatch_df_esp, x='marcaTiempo', y='valor', title='Resumen de datos de series de tiempo ')

fig.update_xaxes(rangeslider_visible=True,)
fig

In [ ]:
cloudwatch_df_esp.head()

In [ ]:
alt.Chart(cloudwatch_df_esp).mark_rect().encode(alt.X('hora:O', title='Hora del dia '),
                                      alt.Y('diaSemana:O', title='diaSemana'),
                                      alt.Color('valor:Q', title='uso del CPU')).properties(
                                            width=800,
                                            height=300)

In [ ]:
alt.Chart(cloudwatch_df_esp).mark_bar().encode(x = 'diaSemana:O',
                                     y = 'valor:Q').properties(width=600)

# 5. Aplicando los Modelos de Aprendizaje no supervisado

### 5.1 Isolation Forests

    Isolation Forest es una método no supervisado para identificar anomalías (outliers)
    cuando los datos no están etiquetados, es decir, no se conoce la clasificación 
    real (anomalía - no anomalía) de las observaciones.

    Eĺ modelo Isolation Forest se obtiene al combinar múltiples isolation tree, cada uno 
    entrenado con una muestra distinta generada por bootstrapping a partir de los datos de 
    originales. El valor predicho para cada observacion es el número de divisiones promedio 
    que se han necesitado para aislar dicha observacion en el conjunto de árboles. Cuanto 
    menor es este valor, mayor es la probabilidad de que se trate de una anomalía. 
    
    
  #### Algoritmo Isolation Tree

        1)Crear un nodo raíz que contiene las N observaciones de entrenamiento.

        2)Seleccionar aleatoriamente un atributo i y un valor aleatorio a dentro del rango observado de i

        3)Crear dos nuevos nodos separando las observaciones acorde al criterio xi≤a o xi>a

        4)Repetir los pasos 2 y 3 hasta que todas las observaciones quedan aisladas de forma individual en nodos terminales.
    
    

In [ ]:
x = cloudwatch_df_esp['valor'].apply(lambda x: [x]).to_list()

iso_forest = IsolationForest(n_estimators = 100, 
                        max_samples = "auto",
                        contamination = 0.01, 
                        random_state = 42)
iso_forest.fit(x)
y_pred = iso_forest.predict(x)
y_pred = [1 if x == -1 else 0 for x in y_pred]
y_pred[:10]
# Points that are 1 are outliers

In [ ]:
cloudwatch_df_esp["anomalias"] = y_pred
cloudwatch_df_esp.head()

In [ ]:
iso_anomaly_df = pd.DataFrame(cloudwatch_df_esp)
iso_anomaly_df = iso_anomaly_df.loc[iso_anomaly_df['anomalias'] == 1]
iso_anomaly_df.head()

In [ ]:
fig = px.line(cloudwatch_df_esp, x='marcaTiempo', y='valor', title='Detección de anomalías no supervisada en la utilización de la CPU ')
fig.add_trace(go.Scatter(x=iso_anomaly_df["marcaTiempo"].to_list(), y=iso_anomaly_df["valor"].to_list(), mode='markers', name='anomalias'))
fig.update_xaxes(rangeslider_visible=True)
fig

Por culpa

## 5.2 Local Outlier Factor

    Es un método de detección de anomalías no supervisado que calcula la desviación de densidad local de un punto 
    de datos dado con respecto a sus vecinos. Considera como valores atípicos las muestras que tienen una densidad 
    sustancialmente más baja que sus vecinas.
    
    El número de vecinos considerados (parámetro n_vecinos)se establece típicamente 
    
    1) mayor que el número mínimo de muestras que un cúmulo tiene que contener,de modo que otras muestras pueden ser valores atípicos locales en relación con este cúmulo,y 
    
    2) menor que el número máximo de muestras cercanas que pueden ser potencialmente valores atípicos locales.En la práctica,por lo general no se dispone de esa información,y la toma de n_vecinos=20 parece funcionar bien en general.

In [ ]:
from sklearn.neighbors import LocalOutlierFactor
lof = LocalOutlierFactor(n_neighbors=2)
y_pred = lof.fit_predict(x)
y_pred = [1 if x == -1 else 0 for x in y_pred]
y_pred[:10]

In [ ]:
cloudwatch_df_esp["anomalias"] = y_pred
cloudwatch_df_esp.head()

In [ ]:
lof_anomaly_df = pd.DataFrame(cloudwatch_df_esp)
lof_anomaly_df = lof_anomaly_df.loc[lof_anomaly_df['anomalias'] == 1]
lof_anomaly_df.head()

In [ ]:
fig = px.line(cloudwatch_df_esp, x='marcaTiempo', y='valor', title='Detección de anomalías no supervisada en la utilización de la CPU ')
fig.add_trace(go.Scatter(x=lof_anomaly_df["marcaTiempo"].to_list(), y=lof_anomaly_df["valor"].to_list(), mode='markers', name='anomalias'))
fig.update_xaxes(rangeslider_visible=True)
fig

# 6. Comparación de Modelos

In [ ]:
fig = px.line(cloudwatch_df_esp, x='marcaTiempo', y='valor', title='Detección de anomalías no supervisada en la utilización de la CPU')
fig.add_trace(go.Scatter(x=lof_anomaly_df["marcaTiempo"].to_list(), y=lof_anomaly_df["valor"].to_list(), mode='markers', name='Local Outlier Factor'))
fig.add_trace(go.Scatter(x=iso_anomaly_df["marcaTiempo"].to_list(), y=iso_anomaly_df["valor"].to_list(), mode='markers', name='Isolation Forests'))
fig.update_xaxes(rangeslider_visible=True)
fig

## Basándose solo en las dos visualizaciones, es fácil ver que Isolation Forest proporciona un modelo más robusto para capturar anomalías. 
## Isolation Forest también permite una mayor flexibilidad al definir un parámetro de contaminación al definir el modelo. 

# Referencias:
### https://www.kaggle.com/yiehyuheng/unsupervised-anomaly-detection-lof-iforest
### https://www.cienciadedatos.net/documentos/66_deteccion_anomalias_isolationforest.html
### https://runebook.dev/es/docs/scikit_learn/auto_examples/neighbors/plot_lof_outlier_detection

# 